# Demonstration of EFlowCalc's API

This notebook contains a simple example of the usage of the API of `EFlowCalc` to calculate streamflow characteristics from streamflow timeseries.

## 1. Load observed and simulated streamflow timeseries

An example file is provided in the folder `examples/` in order for anyone to reproduce this tutorial. Because this is a NetCDF file, we are going to use the Python package `netCDF4`, but EFlowCalc is independent of the file format you are working with because it only requires numpy arrays as inputs for streamflow timeseries and datetime series. The datetime series must be made of `datetime` objects from the `datetime` package.

In [ ]:
from netCDF4 import Dataset
from datetime import datetime, timedelta

# load the timestamp and streamflow timeseries
with Dataset('examples/catchment.sim.flow.nc', 'r', format='NETCDF4') as f:  # read the NetCDF file
    my_flow = f.variables['flow'][:]  # these are the streamflow timeseries
    my_ts = f.variables['DateTime'][:]  # this is the timestamp series for the period

# convert the UNIX Epoch time (contained in the NetCDF file) to datetime objects
my_dt = np.array([datetime(1970, 1, 1) + timedelta(seconds=tstamp) for tstamp in my_ts]) 

## 2. Calculate one or more streamflow charateristics

Now that the dataset is loaded in memory, it is time to use EFlowCalc to calculate the streamflow characteristics from the hydrograph(s). To do so, import `eflowcalc`, which will give you access to the `calculator` Python function as well as all streamflow characteristics implemented in EFlowCalc (as Python functions as well). To calculate any characteristic from your streamflow timeseries, you only need to call the calculator function and provide four positional arguments: the Python function (or an iterable containing several functions) corresponding to the characteristic(s) to be calculated (e.g. `ma41`, `dh4`, `(ma41, dh4)`, etc.), the DateTime numpy array that represents the period the flows are provided for (it can only contain one timeseries), the numpy array containing the streamflow timeseries (it can be multiple timeseries), and finally the catchment drainage area in square kilometers. See below examples using these requirements:

In [ ]:
from eflowcalc import *

# calculate only one streamflow characteristics (e.g. MA41 here)
my_ma41 = calculator(ma41, my_flow, my_dt, 1246)

# calculate multiple streamflow characteristics at once (e.g. MA41, DH4, and RA7 here)
my_sfc = calculator((ma41, dh4, ra7), my_flow, my_dt, 1246)

It is important to be aware that EFlowCalc requires strictly continuous timeseries of streamflow. Moreover, all streamflow characteristics are only computed on full hydrological years (it will automatically trim the head and tail of the timeseries to guarantee so). By default, a hydrological year starts on the 1st of October, which is commonly the case in the Northern hemisphere. However, it can be changed to suit any location using the keyword argument `hydro_year`. For example, if working on a catchment in the Southern hemisphere, it is likely that the hydrological year starts on the 1st of July:

In [ ]:
# calculate only one streamflow characteristics (e.g. MA41 here) for a catchment in the Southern hemisphere
my_ma41 = calculator(ma41, my_flow, my_dt, 1246, water_year='01/07')

EFlowCalc can only deal with 1-dimensional or 2-dimensional numpy arrays as input. EFlowCalc can evaluate multiple simulated timeseries at the same time (using vectorised calculations). This means that the streamflow timeseries can be a 2-dimensional array with both dimensions of size greater than 1, one of them being the time dimension and must match the length of the DateTime series. By default, EFlowCalc is expecting the time dimension to be on the second axis (i.e. `axis=1`) if a 2-dimensional array is provided (applicable to both the streamflow and datetime series). If this is not the case with your dataset, you need to redefine the `axis` keyword argument default value and set it to `0` (for the first axis). Alternatively, you can transpose your arrays before giving them to EFlowCalc.